# Gesture Recognition

### Problem Statement
Imagine you are working as a data scientist at a home electronics company which manufactures state of the art smart televisions. You want to develop a cool feature in the smart-TV that can recognise five different gestures performed by the user which will help users control the TV without using a remote.

The gestures are continuously monitored by the webcam mounted on the TV. Each gesture corresponds to a specific command:

Gesture	Corresponding Action:\
     1. Thumbs Up	Increase the volume.\
     2. Thumbs Down	Decrease the volume.\
     3. Left Swipe	'Jump' backwards 10 seconds.\
     4. Right Swipe	'Jump' forward 10 seconds.\
     5. Stop	Pause the movie.

Each video is a sequence of 30 frames (or images).

### Importing required libraries

In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize as imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('train.csv').readlines())
val_doc = np.random.permutation(open('val.csv').readlines())
batch_size = 50#experiment with the batch size

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

### Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    batch_data[folder,idx] = temp #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

        

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = 'train'
val_path = 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras

In [8]:
# First try with x, y, z
classes, channel = 5, 3 
x, y, z = 30, 120, 120

In [8]:
#write your model here
model_1 = Sequential()

model_1.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(32, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(64, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_1.add(Flatten())

model_1.add(Dense(1000, activation='relu'))
model_1.add(Dropout(0.5))

model_1.add(Dense(500, activation='relu'))
model_1.add(Dropout(0.5))

#softmax layer
model_1.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [9]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization (BatchN  (None, 30, 120, 120, 8)  32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 15, 60, 60, 8)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_1 (Batc  (None, 15, 60, 60, 16)   6

C:\Users\ASUS\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [9]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [10]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [13]:
model_1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_1648\2273047050.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  train ; batch size = 50


C:\Users\ASUS\AppData\Local\Temp\ipykernel_1648\974584729.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Epoch 1/10
12/14 [========================>.....] - ETA: 2:17 - loss: 4.7289 - categorical_accuracy: 0.2783Batch:  14 Index: 50


C:\Users\ASUS\AppData\Local\Temp\ipykernel_1648\974584729.py:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


14/14 [==============================] - ETA: 0s - loss: 4.5124 - categorical_accuracy: 0.2805  Source path =  val ; batch size = 50

Epoch 1: saving model to model_init_2023-02-2007_33_26.620794\model-00001-4.51245-0.28054-2.25369-0.23000.h5
14/14 [==============================] - 1065s 76s/step - loss: 4.5124 - categorical_accuracy: 0.2805 - val_loss: 2.2537 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/10
14/14 [==============================] - ETA: 0s - loss: 2.1516 - categorical_accuracy: 0.4011 
Epoch 2: saving model to model_init_2023-02-2007_33_26.620794\model-00002-2.15161-0.40110-1.28199-0.49000.h5
14/14 [==============================] - 222s 16s/step - loss: 2.1516 - categorical_accuracy: 0.4011 - val_loss: 1.2820 - val_categorical_accuracy: 0.4900 - lr: 0.0010
Epoch 3/10
14/14 [==============================] - ETA: 0s - loss: 1.9413 - categorical_accuracy: 0.4066 
Epoch 3: saving model to model_init_2023-02-2007_33_26.620794\model-00003-1.94130-0.40659-1.35224

# Model 2 -> experimenting with different CNN layer architect**ure**

In [10]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

# Define model 2
model_2 = Sequential()
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Flatten())
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(classes, activation='softmax'))

model_2.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 120, 120, 32)  2624      
                                                                 
 activation (Activation)     (None, 30, 120, 120, 32)  0         
                                                                 
 conv3d_1 (Conv3D)           (None, 30, 120, 120, 32)  27680     
                                                                 
 activation_1 (Activation)   (None, 30, 120, 120, 32)  0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 10, 40, 40, 32)   0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 10, 40, 40, 32)    0         
                                                        

In [11]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14360\876195745.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14360\974584729.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Source path =  train ; batch size = 50
Epoch 1/10


# Model_3: Experimenting with x, y, z values

In [13]:
x, y, z = 30, 60, 60

In [14]:
# Define model b
model_3 = Sequential()
model_3.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_3.add(Activation('relu'))
model_3.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_3.add(Dropout(0.25))

model_3.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_3.add(Activation('relu'))
model_3.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_3.add(Dropout(0.25))

model_3.add(Flatten())
model_3.add(Dense(512, activation='relu'))
model_3.add(Dropout(0.5))
model_3.add(Dense(classes, activation='softmax'))

model_3.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 60, 60, 32)    2624      
                                                                 
 activation (Activation)     (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_1 (Conv3D)           (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_1 (Activation)   (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 10, 20, 20, 32)   0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 10, 20, 20, 32)    0         
                                                        

In [18]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_12936\1978509317.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
C:\Users\ASUS\AppData\Local\Temp\ipykernel_12936\974584729.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Source path =  train ; batch size = 50
Epoch 1/10


# With batch size as 50 we see in above cases that machine is not working, Therefore experimenting with batch size.
# lets take batch size: 30


In [15]:
train_generator = generator(train_path, train_doc, 30)
val_generator = generator(val_path, val_doc, 30)

In [16]:
model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_4704\1978509317.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
C:\Users\ASUS\AppData\Local\Temp\ipykernel_4704\974584729.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Source path =  train ; batch size = 30
Epoch 1/10
14/14 [==============================] - ETA: 0s - loss: 1.7547 - categorical_accuracy: 0.2238 Source path =  val ; batch size = 30

Epoch 1: saving model to model_init_2023-02-2011_16_20.930504\model-00001-1.75468-0.22381-1.60816-0.20000.h5
14/14 [==============================] - 673s 48s/step - loss: 1.7547 - categorical_accuracy: 0.2238 - val_loss: 1.6082 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 2/10
 7/14 [==============>...............] - ETA: 5:02 - loss: 1.6095 - categorical_accuracy: 0.1810Batch:  23 Index: 30


C:\Users\ASUS\AppData\Local\Temp\ipykernel_4704\974584729.py:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


14/14 [==============================] - ETA: 0s - loss: 1.6023 - categorical_accuracy: 0.1938 Batch:  4 Index: 30

Epoch 2: saving model to model_init_2023-02-2011_16_20.930504\model-00002-1.60235-0.19380-1.64583-0.15000.h5
14/14 [==============================] - 374s 25s/step - loss: 1.6023 - categorical_accuracy: 0.1938 - val_loss: 1.6458 - val_categorical_accuracy: 0.1500 - lr: 0.0010
Epoch 3/10
14/14 [==============================] - ETA: 0s - loss: 1.6378 - categorical_accuracy: 0.2143
Epoch 3: saving model to model_init_2023-02-2011_16_20.930504\model-00003-1.63781-0.21429-1.61345-0.20000.h5

Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
14/14 [==============================] - 68s 5s/step - loss: 1.6378 - categorical_accuracy: 0.2143 - val_loss: 1.6134 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 4/10
14/14 [==============================] - ETA: 0s - loss: 1.6121 - categorical_accuracy: 0.1429
Epoch 4: saving model to model_init_2023-02

# Model experiment 4

In [15]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3
nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]
input_shape=(x,y,z,channel)

# Define model
model_4 = Sequential()

model_4.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

model_4.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

model_4.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

model_4.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_4.add(Activation('relu'))
model_4.add(Dropout(0.25))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_4.add(Flatten())

model_4.add(Dense(nb_dense[0], activation='relu'))
model_4.add(Dropout(0.5))

model_4.add(Dense(nb_dense[1], activation='relu'))
model_4.add(Dropout(0.5))

#softmax layer
model_4.add(Dense(nb_dense[2], activation='softmax'))
model_4.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_4.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_3 (Batc  (None, 30, 120, 120, 8)  32        
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 15, 60, 60, 8)    0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_4 (Batc  (None, 15, 60, 60, 16)  

In [16]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [17]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [18]:
batch_size = 10
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
num_epochs = 10
model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_9224\812619276.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
C:\Users\ASUS\AppData\Local\Temp\ipykernel_9224\974584729.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Source path =  train ; batch size = 10
Epoch 1/10
65/67 [============================>.] - ETA: 20s - loss: 2.9781 - categorical_accuracy: 0.1923Batch:  67 Index: 10


C:\Users\ASUS\AppData\Local\Temp\ipykernel_9224\974584729.py:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


67/67 [==============================] - ETA: 0s - loss: 2.9836 - categorical_accuracy: 0.1931 Source path =  val ; batch size = 10

Epoch 1: saving model to model_init_2023-02-1920_31_44.623438\model-00001-2.98365-0.19306-1.61024-0.19000.h5
67/67 [==============================] - 751s 11s/step - loss: 2.9836 - categorical_accuracy: 0.1931 - val_loss: 1.6102 - val_categorical_accuracy: 0.1900 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.9955 - categorical_accuracy: 0.1741
Epoch 2: saving model to model_init_2023-02-1920_31_44.623438\model-00002-2.99548-0.17413-1.62140-0.19000.h5
67/67 [==============================] - 284s 4s/step - loss: 2.9955 - categorical_accuracy: 0.1741 - val_loss: 1.6214 - val_categorical_accuracy: 0.1900 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 2.9322 - categorical_accuracy: 0.1990
Epoch 3: saving model to model_init_2023-02-1920_31_44.623438\model-00003-2.93221-0.19900-1.62137-0.23

# After few more experiments, Final model is below.

In [ ]:
x, y, z = 30, 120, 120

In [ ]:
# First try with x, y, z
classes, channel = 5, 3 

In [ ]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(30,120,120,3)

# Define model
model_final = Sequential()

model_final.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(Activation('relu'))
model_final.add(Dropout(0.25))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_final.add(Flatten())

model_final.add(Dense(nb_dense[0], activation='relu'))
model_final.add(Dropout(0.5))

model_final.add(Dense(nb_dense[1], activation='relu'))
model_final.add(Dropout(0.5))

#softmax layer
model_final.add(Dense(nb_dense[2], activation='softmax'))

In [4]:
optimiser = keras.optimizers.Adam() #write your optimizer
model_final.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_final.summary())

NameError: name 'keras' is not defined

In [40]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [41]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [42]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [43]:
batch_size = 10
num_epochs = 20
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_9224\1713692469.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  train ; batch size = 32


C:\Users\ASUS\AppData\Local\Temp\ipykernel_9224\974584729.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Epoch 1/20
19/21 [==========================>...] - ETA: 1:25 - loss: 2.2074 - categorical_accuracy: 0.2286Batch:  21 Index: 32


C:\Users\ASUS\AppData\Local\Temp\ipykernel_9224\974584729.py:38: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


21/21 [==============================] - ETA: 0s - loss: 2.1547 - categorical_accuracy: 0.2338 Source path =  val ; batch size = 32
Batch:  4 Index: 32

Epoch 1: saving model to model_init_2023-02-1920_31_44.623438\model-00001-2.15471-0.23379-1.59499-0.27000.h5
21/21 [==============================] - 979s 47s/step - loss: 2.1547 - categorical_accuracy: 0.2338 - val_loss: 1.5950 - val_categorical_accuracy: 0.2700 - lr: 0.0010
Epoch 2/20
21/21 [==============================] - ETA: 0s - loss: 1.5734 - categorical_accuracy: 0.2609 
Epoch 2: saving model to model_init_2023-02-1920_31_44.623438\model-00002-1.57344-0.26087-1.49530-0.37500.h5
21/21 [==============================] - 474s 23s/step - loss: 1.5734 - categorical_accuracy: 0.2609 - val_loss: 1.4953 - val_categorical_accuracy: 0.3750 - lr: 0.0010
Epoch 3/20
21/21 [==============================] - ETA: 0s - loss: 1.5161 - categorical_accuracy: 0.3208 
Epoch 3: saving model to model_init_2023-02-1920_31_44.623438\model-00003-1.516

# Experimenting one more time with batch_size and num_epochs

In [44]:
batch_size = 32
num_epochs = 20
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_9224\3618232084.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
C:\Users\ASUS\AppData\Local\Temp\ipykernel_9224\974584729.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)


Epoch 1/20
21/21 [==============================] - ETA: 0s - loss: 0.4058 - categorical_accuracy: 0.8487 
Epoch 1: saving model to model_init_2023-02-1920_31_44.623438\model-00001-0.40579-0.84874-0.97581-0.68750.h5
21/21 [==============================] - 353s 17s/step - loss: 0.4058 - categorical_accuracy: 0.8487 - val_loss: 0.9758 - val_categorical_accuracy: 0.6875 - lr: 2.5000e-04
Epoch 2/20
21/21 [==============================] - ETA: 0s - loss: 0.3552 - categorical_accuracy: 0.8824 
Epoch 2: saving model to model_init_2023-02-1920_31_44.623438\model-00002-0.35519-0.88235-0.71639-0.62500.h5
21/21 [==============================] - 336s 16s/step - loss: 0.3552 - categorical_accuracy: 0.8824 - val_loss: 0.7164 - val_categorical_accuracy: 0.6250 - lr: 2.5000e-04
Epoch 3/20
21/21 [==============================] - ETA: 0s - loss: 0.3155 - categorical_accuracy: 0.8908 
Epoch 3: saving model to model_init_2023-02-1920_31_44.623438\model-00003-0.31547-0.89076-0.67537-0.75000.h5
21/21 [=